In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os 
from pprint import pprint

In [3]:
import numpy as np
import decimal
import copy

In [4]:
import streamm

In this getting started example we will calculate the coupling between P3HT oligomers

Now let's create project and resource to keep track of our work

In [5]:
p3ht_et = streamm.Project('P3HT_ET')

And a resource object to keep track of where our files are 

In [6]:
res_local = streamm.Resource('local')

Update relative location of templates directory 

In [7]:
res_local.dir['templates'] =  os.path.join(res_local.dir['home'],'..','templates','')

In [8]:
res_local.make_dir()

In [9]:
p3ht_et.set_resource(res_local)

Now we need to set a remote resource we have ssh access to and run calculations 

In [10]:
peregrine = streamm.Resource('peregrine')

In [11]:
peregrine.meta['type'] = "ssh"
peregrine.ssh['username'] = "tkemper"    
peregrine.ssh['address'] = "peregrine.hpc.nrel.gov"    
peregrine.dir['storage'] = '/mss/users/%s'%(peregrine.ssh['username'])
peregrine.dir['scratch'] = '/scratch/%s'%(peregrine.ssh['username'])
peregrine.dir['home'] = res_local.dir['home']
peregrine.dir['launch'] = res_local.dir['launch']
peregrine.dir['templates'] = res_local.dir['templates']

# Set default simulation specs
peregrine.properties['allocation'] = 'orgopv'
peregrine.properties['e-mail'] = 'travis.kemper@nrel.gov'
peregrine.properties['scratch'] = peregrine.dir['scratch']
peregrine.properties['allocation'] = 'orgopv'
peregrine.properties['walltime'] = 4
peregrine.properties['nodes'] = int(1)
peregrine.properties['ppn'] = int(24)
peregrine.properties['nproc'] = peregrine.properties['nodes']*peregrine.properties['ppn']
peregrine.properties['queue'] = 'short'
peregrine.properties['feature'] = '24core'
peregrine.properties['exe_command'] = 'qsub '

In [12]:
pprint(peregrine.properties)

{u'allocation': 'orgopv',
 'e-mail': 'travis.kemper@nrel.gov',
 u'exe_command': 'qsub ',
 u'feature': '24core',
 u'nodes': 1,
 u'nproc': 24,
 u'pmem': 1500,
 u'ppn': 24,
 u'queue': 'short',
 'scratch': '/scratch/tkemper',
 u'walltime': 4}


In [13]:
peregrine.make_dir()

Explicitely create a thiophene molecule

In [14]:
bbTh = streamm.Buildingblock('thiophene')
symbols = ['C','C','C','C','S','H','H','H','H']
positions = [ ]
positions.append([-1.55498576,-1.91131218,-0.00081000])
positions.append([-0.17775976,-1.91131218,-0.00081000])
positions.append([0.34761524,-0.57904218,-0.00081000])
positions.append([-0.65884476,0.36101082,0.00000000])
positions.append([-2.16948076,-0.35614618,-0.00000800])
positions.append([-2.18966076,-2.79526518,-0.00132100])
positions.append([0.45389024,-2.80145418,-0.00106400])
positions.append([1.41682424,-0.35961818,-0.00138200])
positions.append([-0.51943676,1.44024682,0.00064700])
for i in range(len(symbols)):
    pt_i = streamm.Particle(symbol=symbols[i])
    pos_i = positions[i]
    bbTh.add_partpos(pt_i,pos_i)

In [15]:
bbTh.particles[5].rsite = 'termcap'
bbTh.particles[6].rsite = 'funccap'
bbTh.particles[8].rsite = 'termcap'

Set some properties of the molecule to keep track of the parts

In [16]:
c_cnt =1
h_cnt =1

for pkey_i, particle_i  in bbTh.particles.iteritems():
            if( particle_i.symbol == 'C' ):
                particle_i.label = "C%d"%(c_cnt)
                particle_i.resname = "SCP2"
                particle_i.residue = 1

                c_cnt +=1 
            if( particle_i.symbol == 'S' ):
                particle_i.resname = "ThS"
                particle_i.residue = 2

            if( particle_i.symbol == 'H' ):
                particle_i.label = "H%d"%(h_cnt)
                particle_i.resname = "HA"
                particle_i.residue = 3

                h_cnt +=1 
                

Set the force-field type and guess some reasonable charges 

In [17]:
for pkey_i, particle_i  in bbTh.particles.iteritems():
            if( particle_i.symbol == 'C' ):
                particle_i.paramkey = 'CA'
                particle_i.charge = -0.025
            if( particle_i.symbol == 'S' ):
                particle_i.paramkey = 'S'
                particle_i.charge = -0.3
            if( particle_i.symbol == 'H' ):
                particle_i.paramkey = 'HA'
                particle_i.charge = 0.1

Check molecule is neutral 

In [18]:
total_charge = 0.0
for pkey_i, particle_i  in bbTh.particles.iteritems():
    total_charge += particle_i.charge
print total_charge

-2.77555756156e-17


Optimize structure with NWChem

But let's put it in a function this time

In [19]:
def nw_opt(project_i,bb_i,res_i):
    '''Optimize a streamm Buildingblock object with nwchem 
    
    '''
    nwchem_i = streamm.NWChem('nw_opt_{}'.format(bb_i.tag))
    print nwchem_i.tag 
    # Add thiophene structure 
    nwchem_i.strucC = copy.deepcopy(bb_i)
    # Set calculation to run on external resource
    nwchem_i.set_resource(res_i)
    # Make the local directories 
    nwchem_i.make_dir()
    #Change to the `launch` directory
    os.chdir(nwchem_i.dir['launch'])
    # Copy over templates
    nwchem_i.cp_file('templates','run',"nwchem_peregrine.pbs",'templates','launch')
    nwchem_i.cp_file('templates','nw',"nwchem.nw",'templates','launch')
    # Read in templates files 
    nwchem_i.load_str('templates','nw')        
    nwchem_i.load_str('templates','run')
    # Set calculation properties 
    nwchem_i.properties['basis'] = '6-31g'
    nwchem_i.properties['method'] = 'UHF'
    nwchem_i.properties['charge'] = 0
    nwchem_i.properties['spin_mult'] = 1
    nwchem_i.properties['task'] = 'SCF optimize'
    nwchem_i.properties['coord'] = nwchem_i.strucC.write_coord()
    # 
    pprint(nwchem_i.properties)
    # Replace <key> with properties value 
    nwchem_i.replacewrite_prop('nw','input','nw','%s.nw'%(nwchem_i.tag))
    nwchem_i.properties['input_nw'] = nwchem_i.files['input']['nw']
    nwchem_i.replacewrite_prop('run','scripts','run','%s.pbs'%(nwchem_i.tag))
    #
    nwchem_i.add_file('output','log',"%s.log"%(nwchem_i.tag))
    # Save details in .json files 
    os.chdir(nwchem_i.dir['home'])
    p3ht_et.dump_json()
    # 
    os.chdir(nwchem_i.dir['launch'])
    # 
    nwchem_i.push()
    # 
    nwchem_i.run()
    # Add calculation to project
    project_i.add_calc(nwchem_i,deepcopy = True)
    # 
    return project_i 

In [20]:
p3ht_et = nw_opt(p3ht_et,bbTh,peregrine)

nw_opt_thiophene
{u'allocation': 'orgopv',
 u'basis': '6-31g',
 u'charge': 0,
 'comp_key': 'compressed',
 'compress': 'tar -czf ',
 'compress_sufix': 'tgz',
 'coord': u'     C      -1.55498576      -1.91131218      -0.00081000 \n     C      -0.17775976      -1.91131218      -0.00081000 \n     C       0.34761524      -0.57904218      -0.00081000 \n     C      -0.65884476       0.36101082       0.00000000 \n     S      -2.16948076      -0.35614618      -0.00000800 \n     H      -2.18966076      -2.79526518      -0.00132100 \n     H       0.45389024      -2.80145418      -0.00106400 \n     H       1.41682424      -0.35961818      -0.00138200 \n     H      -0.51943676       1.44024682       0.00064700 \n',
 'e-mail': 'travis.kemper@nrel.gov',
 u'exe_command': 'qsub ',
 u'feature': '24core',
 u'finish_str': u'Total times  cpu:',
 u'maxiter': 100,
 u'method': 'UHF',
 u'nodes': 1,
 u'nproc': 24,
 u'pmem': 1500,
 u'ppn': 24,
 u'queue': 'short',
 'scratch': u'/scratch/tkemper/nw_opt_thiophene/'

In [21]:
nwchem_i = p3ht_et.calculations['nw_opt_thiophene']

Check status unit finished

In [24]:
while( nwchem_i.meta['status'] != 'finished'):
    nwchem_i.check()

In [25]:
print nwchem_i.meta['status']

finished


In [26]:
nwchem_i.analysis()

Print energies 

In [27]:
print nwchem_i.properties['alpha_energies'][10:20]
print nwchem_i.properties['energy']

[-0.4732391, -0.3475018, -0.3290008, 0.1318747, 0.2130991, 0.2315597, 0.2416684, 0.269473, 0.2970013, 0.3198832]
-551.190758149


Check that the positions of the structure have been optimized 

In [28]:
print bbTh.positions

[[ -1.55498576e+00  -1.91131218e+00  -8.10000000e-04]
 [ -1.77759760e-01  -1.91131218e+00  -8.10000000e-04]
 [  3.47615240e-01  -5.79042180e-01  -8.10000000e-04]
 [ -6.58844760e-01   3.61010820e-01   0.00000000e+00]
 [ -2.16948076e+00  -3.56146180e-01  -8.00000000e-06]
 [ -2.18966076e+00  -2.79526518e+00  -1.32100000e-03]
 [  4.53890240e-01  -2.80145418e+00  -1.06400000e-03]
 [  1.41682424e+00  -3.59618180e-01  -1.38200000e-03]
 [ -5.19436760e-01   1.44024682e+00   6.47000000e-04]]


In [29]:
print nwchem_i.strucC.positions

[[ -1.52907081e+00  -1.96211218e+00  -7.82430000e-04]
 [ -1.87413740e-01  -1.91422664e+00  -1.04851000e-03]
 [  3.42641760e-01  -5.70472900e-01  -5.95970000e-04]
 [ -6.05026820e-01   3.80452780e-01  -2.05200000e-05]
 [ -2.25044444e+00  -3.24037990e-01   4.69600000e-05]
 [ -2.16246857e+00  -2.81998700e+00  -1.08746000e-03]
 [  4.30968660e-01  -2.78822534e+00  -1.62085000e-03]
 [  1.39117928e+00  -3.53996750e-01  -7.83750000e-04]
 [ -4.82204160e-01   1.43971341e+00   3.34540000e-04]]


Update positions with optimized geometry 

In [30]:
for pk,p in bbTh.particles.iteritems():
    bbTh.positions[pk] = nwchem_i.strucC.positions[pk]
    print pk,p.symbol,bbTh.positions[pk]

0 C [ -1.52907081e+00  -1.96211218e+00  -7.82430000e-04]
1 C [ -1.87413740e-01  -1.91422664e+00  -1.04851000e-03]
2 C [ 0.34264176 -0.5704729  -0.00059597]
3 C [ -6.05026820e-01   3.80452780e-01  -2.05200000e-05]
4 S [ -2.25044444e+00  -3.24037990e-01   4.69600000e-05]
5 H [ -2.16246857e+00  -2.81998700e+00  -1.08746000e-03]
6 H [  4.30968660e-01  -2.78822534e+00  -1.62085000e-03]
7 H [  1.39117928e+00  -3.53996750e-01  -7.83750000e-04]
8 H [ -4.82204160e-01   1.43971341e+00   3.34540000e-04]


Store the results in a tar ball in the storage directory 

In [31]:
nwchem_i.store()

Now let us calculate the ESP charges to use in our forcefield 

Again let's make it a function

In [32]:
def nw_esp(project_i,bb_i,res_i):
    '''Calculate ESP charges of a streamm Buildingblock object with nwchem 
    
    '''
    nwchem_esp = streamm.NWChem('nw_esp_{}'.format(bb_i.tag))
    print(nwchem_esp.tag)
    # Add thiophene structure with optimized coordinates from previous calculation
    nwchem_esp.strucC = copy.deepcopy(bb_i)
    # Set calculation to run on external resource
    nwchem_esp.set_resource(res_i)
    # Add calculation to project
    project_i.add_calc(nwchem_esp)
    # Make the local directories 
    nwchem_esp.make_dir()
    # Change to the `launch` directory
    os.chdir(nwchem_esp.dir['launch'])
    #
    nwchem_esp.cp_file('templates','run',"nwchem_peregrine.pbs",'templates','launch')
    nwchem_esp.cp_file('templates','nw',"nwchem_esp.nw",'templates','launch')
    #
    nwchem_esp.load_str('templates','nw')        
    nwchem_esp.load_str('templates','run')
    # 
    nwchem_esp.properties['basis'] = '6-31g'
    nwchem_esp.properties['method'] = 'UHF'
    nwchem_esp.properties['charge'] = 0
    nwchem_esp.properties['spin_mult'] = 1
    nwchem_esp.properties['task'] = 'SCF'
    nwchem_esp.properties['coord'] = nwchem_esp.strucC.write_coord()

    pprint(nwchem_esp.properties)

    nwchem_esp.replacewrite_prop('nw','input','nw','%s.nw'%(nwchem_esp.tag))

    nwchem_esp.properties['input_nw'] = nwchem_esp.files['input']['nw']
    nwchem_esp.replacewrite_prop('run','scripts','run','%s.pbs'%(nwchem_esp.tag))

    nwchem_esp.add_file('output','log',"%s.log"%(nwchem_esp.tag))

    # Save details in .json files 

    os.chdir(nwchem_esp.dir['home'])
    project_i.dump_json()

    os.chdir(nwchem_esp.dir['launch'])
    nwchem_esp.push()
    nwchem_esp.run()
    # Add calculation to project
    project_i.add_calc(nwchem_esp,deepcopy = True)
    # 
    return project_i 
    
    

In [33]:
p3ht_et = nw_esp(p3ht_et,bbTh,peregrine)

nw_esp_thiophene
{u'allocation': 'orgopv',
 u'basis': '6-31g',
 u'charge': 0,
 'comp_key': 'compressed',
 'compress': 'tar -czf ',
 'compress_sufix': 'tgz',
 'coord': u'     C      -1.52907081      -1.96211218      -0.00078243 \n     C      -0.18741374      -1.91422664      -0.00104851 \n     C       0.34264176      -0.57047290      -0.00059597 \n     C      -0.60502682       0.38045278      -0.00002052 \n     S      -2.25044444      -0.32403799       0.00004696 \n     H      -2.16246857      -2.81998700      -0.00108746 \n     H       0.43096866      -2.78822534      -0.00162085 \n     H       1.39117928      -0.35399675      -0.00078375 \n     H      -0.48220416       1.43971341       0.00033454 \n',
 'e-mail': 'travis.kemper@nrel.gov',
 u'exe_command': 'qsub ',
 u'feature': '24core',
 u'finish_str': u'Total times  cpu:',
 u'maxiter': 100,
 u'method': 'UHF',
 u'nodes': 1,
 u'nproc': 24,
 u'pmem': 1500,
 u'ppn': 24,
 u'queue': 'short',
 'scratch': u'/scratch/tkemper/nw_esp_thiophene/'

Check status until finished

In [35]:
nwchem_i = p3ht_et.calculations['nw_esp_thiophene']

In [36]:
while( nwchem_i.meta['status'] != 'finished'):
    nwchem_i.check()

In [37]:
p3ht_et.check()

Calculation nw_opt_thiophene has status stored
Calculation nw_esp_thiophene has status finished


In [38]:
nwchem_i.analysis()

In [39]:
total_charge = 0.0 
for pk,p in nwchem_i.strucC.particles.iteritems():
    print pk,p.symbol,p.charge
    total_charge += p.charge
print total_charge

0 C -0.08582
1 C -0.13691
2 C -0.08587
3 C -0.108793
4 S -0.117705
5 H 0.153827
6 H 0.119023
7 H 0.109209
8 H 0.153039
-8.32667268469e-17


Update the charges of the Buildingblock

In [40]:
for pk,p in bbTh.particles.iteritems():
    p.charge = nwchem_i.strucC.particles[pk].charge
    print pk,p.symbol,p.charge

0 C -0.08582
1 C -0.13691
2 C -0.08587
3 C -0.108793
4 S -0.117705
5 H 0.153827
6 H 0.119023
7 H 0.109209
8 H 0.153039


Store the results 

In [41]:
nwchem_i.store()

In [42]:
p3ht_et.check()

Calculation nw_opt_thiophene has status stored
Calculation nw_esp_thiophene has status stored


Build hexane

In [43]:
bbHex = streamm.Buildingblock('hexane')
symbols = ['C','H','H','H','C','H','H','C','H','H','C','H','H','C','H','H','C','H','H','H']
positions = [ ]
positions.append([-6.410969,-0.381641,-0.000031])
positions.append([-7.310084,0.245311,-0.000038])
positions.append([-6.456117,-1.028799,0.884636])
positions.append([-6.456111,-1.028812,-0.884689])
positions.append([-5.135268,0.467175,-0.000033])
positions.append([-5.135484,1.128782,0.877977])
positions.append([-5.135479,1.128771,-0.87805])
positions.append([-3.850566,-0.371258,-0.000024])
positions.append([-3.85112,-1.033978,0.87841])
positions.append([-3.851114,-1.033987,-0.878451])
positions.append([-2.567451,0.469603,-0.000024])
positions.append([-2.567784,1.132155,0.8784])
positions.append([-2.567776,1.132146,-0.878455])
positions.append([-1.283527,-0.370234,-0.000013])
positions.append([-1.28337,-1.032804,0.87836])
positions.append([-1.28336,-1.032812,-0.87838])
positions.append([0.00482234,0.47342231,-0.00000898])
positions.append([0.02595107,1.09220686,0.87266464])
positions.append([0.85585781,-0.17514133,0.00194589])
positions.append([0.02780957,1.08937798,-0.87463473])
for i in range(len(symbols)):
    pt_i = streamm.Particle(symbol=symbols[i])
    pos_i = positions[i]
    bbHex.add_partpos(pt_i,pos_i)

In [44]:
bbHex.particles[0].rsite = 'rg'
bbHex.particles[1].rsite = 'rgcap'

In [45]:
c_cnt =1
h_cnt =1
for pkey_i, particle_i  in bbHex.particles.iteritems():
            if( particle_i.symbol == 'C' ):
                particle_i.label = "C%d"%(c_cnt)
                particle_i.resname = "SCP3"
                particle_i.residue = c_cnt
                c_cnt +=1 
            if( particle_i.symbol == 'H' ):
                particle_i.label = "H%d"%(h_cnt)
                particle_i.resname = "HC"
                particle_i.residue = c_cnt -1 
                h_cnt +=1 

Set the parameter keys and some reasonable atomic charges 

In [46]:
for pkey_i, particle_i  in bbHex.particles.iteritems():
            if( particle_i.symbol == 'C' ):
                particle_i.paramkey = 'CT'
                particle_i.charge = -0.12

            if( particle_i.symbol == 'H' ):
                particle_i.paramkey = 'HC'
                particle_i.charge = 0.06
            print pkey_i, particle_i.symbol,particle_i.charge

0 C -0.12
1 H 0.06
2 H 0.06
3 H 0.06
4 C -0.12
5 H 0.06
6 H 0.06
7 C -0.12
8 H 0.06
9 H 0.06
10 C -0.12
11 H 0.06
12 H 0.06
13 C -0.12
14 H 0.06
15 H 0.06
16 C -0.12
17 H 0.06
18 H 0.06
19 H 0.06


In [47]:
bbHex.particles[0].charge  = -0.18
bbHex.particles[16].charge  = -0.18

In [48]:
total_charge = 0.0
for pkey_i, particle_i  in bbHex.particles.iteritems():
            total_charge += particle_i.charge 

Check that the molecule is neutral 

In [49]:
print total_charge

0.0


Now let us optimze and calculate ESP charges for hexane

Optimize structure with NWChem

In [50]:
p3ht_et = nw_opt(p3ht_et,bbHex,peregrine)

nw_opt_hexane
{u'allocation': 'orgopv',
 u'basis': '6-31g',
 u'charge': 0,
 'comp_key': 'compressed',
 'compress': 'tar -czf ',
 'compress_sufix': 'tgz',
 'coord': u'     C      -6.41096900      -0.38164100      -0.00003100 \n     H      -7.31008400       0.24531100      -0.00003800 \n     H      -6.45611700      -1.02879900       0.88463600 \n     H      -6.45611100      -1.02881200      -0.88468900 \n     C      -5.13526800       0.46717500      -0.00003300 \n     H      -5.13548400       1.12878200       0.87797700 \n     H      -5.13547900       1.12877100      -0.87805000 \n     C      -3.85056600      -0.37125800      -0.00002400 \n     H      -3.85112000      -1.03397800       0.87841000 \n     H      -3.85111400      -1.03398700      -0.87845100 \n     C      -2.56745100       0.46960300      -0.00002400 \n     H      -2.56778400       1.13215500       0.87840000 \n     H      -2.56777600       1.13214600      -0.87845500 \n     C      -1.28352700      -0.37023400      -0.00001

In [51]:
nwchem_i = p3ht_et.calculations['nw_opt_hexane']

In [52]:
while( nwchem_i.meta['status'] != 'finished'):
    nwchem_i.check()

In [53]:
p3ht_et.check()

Calculation nw_opt_thiophene has status stored
Calculation nw_opt_hexane has status finished
Calculation nw_esp_thiophene has status stored


Get the calculation from the project object 

In [54]:
nwchem_i.analysis()

Print energies 

In [55]:
print nwchem_i.properties['alpha_energies'][10:20]
print nwchem_i.properties['energy']

[-0.5549424, -0.5282407, -0.5218841, -0.4996812, -0.4774207, -0.4626707, -0.453462, -0.4506618, -0.4351213, 0.2272358]
-235.271544555


Check that the positions of the structure have been optimized 

In [56]:
for pk,p in bbHex.particles.iteritems():
    print pk,p.symbol,bbHex.positions[pk]

0 C [ -6.41096900e+00  -3.81641000e-01  -3.10000000e-05]
1 H [ -7.31008400e+00   2.45311000e-01  -3.80000000e-05]
2 H [-6.456117 -1.028799  0.884636]
3 H [-6.456111 -1.028812 -0.884689]
4 C [ -5.13526800e+00   4.67175000e-01  -3.30000000e-05]
5 H [-5.135484  1.128782  0.877977]
6 H [-5.135479  1.128771 -0.87805 ]
7 C [ -3.85056600e+00  -3.71258000e-01  -2.40000000e-05]
8 H [-3.85112  -1.033978  0.87841 ]
9 H [-3.851114 -1.033987 -0.878451]
10 C [ -2.56745100e+00   4.69603000e-01  -2.40000000e-05]
11 H [-2.567784  1.132155  0.8784  ]
12 H [-2.567776  1.132146 -0.878455]
13 C [ -1.28352700e+00  -3.70234000e-01  -1.30000000e-05]
14 H [-1.28337  -1.032804  0.87836 ]
15 H [-1.28336  -1.032812 -0.87838 ]
16 C [  4.82234000e-03   4.73422310e-01  -8.98000000e-06]
17 H [ 0.02595107  1.09220686  0.87266464]
18 H [ 0.85585781 -0.17514133  0.00194589]
19 H [ 0.02780957  1.08937798 -0.87463473]


In [58]:
print nwchem_i.strucC.positions

[[ -6.40750402e+00  -3.78933400e-01   3.82600000e-05]
 [ -7.29449606e+00   2.45125210e-01   7.55700000e-05]
 [ -6.45189733e+00  -1.01748730e+00   8.76518070e-01]
 [ -6.45196494e+00  -1.01746560e+00  -8.76454320e-01]
 [ -5.13183582e+00   4.68072510e-01  -2.06100000e-05]
 [ -5.13085507e+00   1.11976646e+00   8.70266420e-01]
 [ -5.13100496e+00   1.11992701e+00  -8.70176510e-01]
 [ -3.85234174e+00  -3.76633680e-01  -1.66770000e-04]
 [ -3.85437422e+00  -1.02906800e+00   8.70770400e-01]
 [ -3.85456733e+00  -1.02900551e+00  -8.71165270e-01]
 [ -2.57086460e+00   4.64371440e-01  -3.23330000e-04]
 [ -2.56880479e+00   1.11697554e+00   8.70498840e-01]
 [ -2.56863934e+00   1.11653225e+00  -8.71457780e-01]
 [ -1.29163291e+00  -3.80800580e-01   1.34730000e-04]
 [ -1.29309539e+00  -1.03225153e+00   8.70587370e-01]
 [ -1.29281926e+00  -1.03278775e+00  -8.69937750e-01]
 [ -1.52777000e-02   4.65255070e-01   7.81100000e-05]
 [  2.92141100e-02   1.10465287e+00   8.75954170e-01]
 [  8.71153410e-01  -1.59617

Update positions in Buildingblock object

In [59]:
for pk,p in bbHex.particles.iteritems():
    bbHex.positions[pk] = nwchem_i.strucC.positions[pk]
    print pk,p.symbol,bbHex.positions[pk]

0 C [ -6.40750402e+00  -3.78933400e-01   3.82600000e-05]
1 H [ -7.29449606e+00   2.45125210e-01   7.55700000e-05]
2 H [-6.45189733 -1.0174873   0.87651807]
3 H [-6.45196494 -1.0174656  -0.87645432]
4 C [ -5.13183582e+00   4.68072510e-01  -2.06100000e-05]
5 H [-5.13085507  1.11976646  0.87026642]
6 H [-5.13100496  1.11992701 -0.87017651]
7 C [ -3.85234174e+00  -3.76633680e-01  -1.66770000e-04]
8 H [-3.85437422 -1.029068    0.8707704 ]
9 H [-3.85456733 -1.02900551 -0.87116527]
10 C [ -2.57086460e+00   4.64371440e-01  -3.23330000e-04]
11 H [-2.56880479  1.11697554  0.87049884]
12 H [-2.56863934  1.11653225 -0.87145778]
13 C [ -1.29163291e+00  -3.80800580e-01   1.34730000e-04]
14 H [-1.29309539 -1.03225153  0.87058737]
15 H [-1.29281926 -1.03278775 -0.86993775]
16 C [ -1.52777000e-02   4.65255070e-01   7.81100000e-05]
17 H [ 0.02921411  1.10465287  0.87595417]
18 H [ 0.87115341 -0.15961795  0.00137754]
19 H [ 0.03046875  1.10285675 -0.87703532]


Store the results in a tar ball in the storage directory 

In [61]:
nwchem_i.store()

Now let us calculate the ESP charges to use in our forcefield 

In [62]:
p3ht_et = nw_esp(p3ht_et,bbHex,peregrine)

nw_esp_hexane
{u'allocation': 'orgopv',
 u'basis': '6-31g',
 u'charge': 0,
 'comp_key': 'compressed',
 'compress': 'tar -czf ',
 'compress_sufix': 'tgz',
 'coord': u'     C      -6.40750402      -0.37893340       0.00003826 \n     H      -7.29449606       0.24512521       0.00007557 \n     H      -6.45189733      -1.01748730       0.87651807 \n     H      -6.45196494      -1.01746560      -0.87645432 \n     C      -5.13183582       0.46807251      -0.00002061 \n     H      -5.13085507       1.11976646       0.87026642 \n     H      -5.13100496       1.11992701      -0.87017651 \n     C      -3.85234174      -0.37663368      -0.00016677 \n     H      -3.85437422      -1.02906800       0.87077040 \n     H      -3.85456733      -1.02900551      -0.87116527 \n     C      -2.57086460       0.46437144      -0.00032333 \n     H      -2.56880479       1.11697554       0.87049884 \n     H      -2.56863934       1.11653225      -0.87145778 \n     C      -1.29163291      -0.38080058       0.00013

Check status unit finished

In [64]:
nwchem_i = p3ht_et.calculations['nw_esp_hexane']

In [65]:
while( nwchem_i.meta['status'] != 'finished'):
    nwchem_i.check()

In [66]:
p3ht_et.check()

Calculation nw_opt_thiophene has status stored
Calculation nw_opt_hexane has status stored
Calculation nw_esp_thiophene has status stored
Calculation nw_esp_hexane has status finished


In [67]:
nwchem_i.analysis()

In [68]:
for pk,p in nwchem_i.strucC.particles.iteritems():
    print pk,p.symbol,p.charge

0 C -0.104962
1 H 0.014483
2 H 0.011985
3 H 0.013446
4 C 0.193663
5 H -0.05557
6 H -0.050715
7 C 0.019904
8 H -0.021599
9 H -0.023266
10 C 0.038815
11 H -0.031314
12 H -0.028887
13 C 0.205648
14 H -0.055581
15 H -0.059858
16 C -0.094065
17 H 0.006369
18 H 0.008395
19 H 0.013109


In [69]:
total_charge = 0.0 
for pk,p in nwchem_i.strucC.particles.iteritems():
    total_charge += p.charge
print total_charge

-2.25514051877e-17


Print energies 

In [70]:
print nwchem_i.properties['alpha_energies'][10:20]
print nwchem_i.properties['energy']

[-0.5549412, -0.5259099, -0.525462, -0.5002824, -0.4784776, -0.4652361, -0.4549634, -0.4542151, -0.4338154, 0.228061]
-235.271544113


Update the charges of the Buildingblock

In [71]:
for pk,p in bbHex.particles.iteritems():
    p.charge = nwchem_i.strucC.particles[pk].charge
    print pk,p.symbol,p.charge

0 C -0.104962
1 H 0.014483
2 H 0.011985
3 H 0.013446
4 C 0.193663
5 H -0.05557
6 H -0.050715
7 C 0.019904
8 H -0.021599
9 H -0.023266
10 C 0.038815
11 H -0.031314
12 H -0.028887
13 C 0.205648
14 H -0.055581
15 H -0.059858
16 C -0.094065
17 H 0.006369
18 H 0.008395
19 H 0.013109


Store the results 

In [72]:
nwchem_i.store()

Now we have optimized geometries and ESP Charges for each buildingblock

So let us make some P3HT oligomers 

In [88]:
res_local.make_dir()

In [90]:
os.chdir(res_local.dir['materials']) 

First we need to identify the bonding within the Buildingblock

In [73]:
bbTh.bonded_nblist = bbTh.guess_nblist(0,radii_buffer=1.35)

In [74]:
bbTh.bonded_bonds()
bbTh.bonded_angles()
bbTh.bonded_dih()

In [75]:
bbHex.bonded_nblist = bbHex.guess_nblist(0,radii_buffer=1.35)

In [76]:
bbHex.bonded_bonds()
bbHex.bonded_angles()
bbHex.bonded_dih()

In [77]:
bbTh.find_rsites()
bbHex.find_rsites()

In [78]:
print(bbTh.show_rsites())

rsite:termcap[ paticle:atom[5] H1 (H) index:5 n_bonds:1] 
rsite:termcap[ paticle:atom[8] H4 (H) index:8 n_bonds:1] 
rsite:funccap[ paticle:atom[6] H2 (H) index:6 n_bonds:1] 



In [79]:
print(bbHex.show_rsites())

rsite:rg[ paticle:atom[0] C1 (C) index:0 n_bonds:4] 
rsite:rgcap[ paticle:atom[1] H1 (H) index:1 n_bonds:1] 



In [81]:
import streamm.structures.buildingblock as bb

In [82]:
ht = bb.attach(bbTh,bbHex,'funccap',0,'rgcap',0,tag='3-hexyl-thiophene')

Check that the molecule looks good

In [91]:
ht.write_xyz()

Check the charges of the removed hydrogens got summed onto the functionalized carbons correctly

In [101]:
ht.calc_charge()
ht.charge

-2.2551405187698492e-17

In [92]:
print(ht.show_rsites())

rsite:termcap[ paticle:atom[5] H1 (H) index:5 n_bonds:1] 
rsite:termcap[ paticle:atom[7] H4 (H) index:7 n_bonds:1] 
rsite:rg[ paticle:atom[8] C1 (C) index:8 n_bonds:4] 



Now that we have the monomer let us set up the force field parameters 

Our potential is in kCal/mol (`kCalmol`) so let's get the unit dictionary and create our own default

Like the force-field example we will use the oplsaa force-field

In [198]:
os.chdir(res_local.dir['home']) 

In [199]:
from pathlib2 import Path

In [200]:
need_files = ['oplsaa.pkl']
for f in need_files:
    path = Path(f)
    if not path.is_file():
        print("Need to run forcefields.ipynb")
        os.system("jupyter nbconvert --to python  forcefields.ipynb")
        os.system("python forcefields.py")

In [201]:
oplsaa = streamm.forcefields.parameters.read_pickle('oplsaa')

In [202]:
print oplsaa.unit_conf['energy']

kCalmol


We need to add the conjugated carbons, hydrogen and sulfur atom types 

In [203]:
import streamm.forcefields.particletype as particletype

In [204]:
import pymatgen_core.core.periodic_table as periodic_table

Set some parameters from J. Am. Chem. Soc., 1996, 118 (45), pp 11225–11236

In [205]:
CA = particletype.Particletype('CA')
HA = particletype.Particletype('HA')

In [206]:
CA.update_units(oplsaa.unit_conf)
HA.update_units(oplsaa.unit_conf)

In [207]:
CA.epsilon = 0.070 # kcal/mol
CA.sigma = 3.55 # Angstroms 

In [208]:
HA.epsilon = 0.030 # kcal/mol
HA.sigma = 2.42 # Angstroms 

In [209]:
CA.mass =  periodic_table.Element['C'].atomic_mass.real
HA.mass =  periodic_table.Element['H'].atomic_mass.real

In [210]:
print CA,HA

 CA epsilon:0.07 sigma:3.55  HA epsilon:0.03 sigma:2.42


In [211]:
S = particletype.Particletype('S')

In [212]:
S.update_units(oplsaa.unit_conf)

Set some parameters from J. Am. Chem. Soc., 1996, 118 (45), pp 11225–11236

In [213]:
S.epsilon = 0.25 # kcal/mol
S.sigma = 3.55 # Angstroms 

In [214]:
S.mass =  periodic_table.Element['S'].atomic_mass.real

Add to forcefield parameters container

In [215]:
oplsaa.add_particletype(CA)
oplsaa.add_particletype(HA)
oplsaa.add_particletype(S)

Set the bond stretching parameters 

In [216]:
import streamm.forcefields.bondtype as bondtype

In [217]:
CA_HA = bondtype.Bondtype('CA','HA',unit_conf=oplsaa.unit_conf)
CT_CA = bondtype.Bondtype('CT','CA',unit_conf=oplsaa.unit_conf)
S_CA = bondtype.Bondtype('S','CA',unit_conf=oplsaa.unit_conf)

In [218]:
CA_HA.setharmonic(1.080,367.0)
CT_CA.setharmonic(1.51,317.0)
S_CA.setharmonic(1.71,250.0)

In [219]:
import streamm.forcefields.angletype as angletype

In [220]:
CA_CA_CA = angletype.Angletype('CA','CA','CA',unit_conf=oplsaa.unit_conf)
CA_CA_HA = angletype.Angletype('CA','CA','HA',unit_conf=oplsaa.unit_conf)
CA_S_CA = angletype.Angletype('CA','S','CA',unit_conf=oplsaa.unit_conf)
S_CA_HA = angletype.Angletype('S','CA','HA',unit_conf=oplsaa.unit_conf)
S_CA_CA = angletype.Angletype('S','CA','CA',unit_conf=oplsaa.unit_conf)
CA_CA_CT = angletype.Angletype('CA','CA','CT',unit_conf=oplsaa.unit_conf)
HA_CA_CT = angletype.Angletype('HA','CA','CT',unit_conf=oplsaa.unit_conf)
CA_CT_HC = angletype.Angletype('CA','CT','HC',unit_conf=oplsaa.unit_conf)
CA_CT_CT = angletype.Angletype('CA','CT','CT',unit_conf=oplsaa.unit_conf)
CT_CT_HC = angletype.Angletype('CT','CT','HA',unit_conf=oplsaa.unit_conf)

In [221]:
CA_CA_CA.setharmonic(63.0,120.0)
CA_CA_HA.setharmonic(35.0,120.0)
CA_S_CA.setharmonic(70.0,92.2)
S_CA_CA.setharmonic(70.0,111.0)
S_CA_HA.setharmonic(35.0,120.0)
CA_CA_CT.setharmonic(70.0,120.0)
HA_CA_CT.setharmonic(35.0,120.0)
CA_CT_HC.setharmonic(50.0,109.5)
CA_CT_CT.setharmonic(63.0,114.0)
CT_CT_HC.setharmonic(50.0,109.5)

In [222]:
import streamm.forcefields.dihtype as dihtype

In [223]:
# del oplsaa.dihtypes[2]

In [224]:
dih_i = dihtype.Dihtype('X','CT','CT','X',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(1.74,-0.157,0.0,0.0)
oplsaa.add_dihtype(dih_i)

In [225]:
dih_i = dihtype.Dihtype('X','CA','CA','X',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,1.33,0.0,0.0)
oplsaa.add_dihtype(dih_i)

In [226]:
dih_i = dihtype.Dihtype('S','CA','CA','HA',unit_conf=oplsaa.unit_conf)
dih_i.type ='opls'
dih_i.setopls(0.0,10.75,0.0,0.0)
oplsaa.add_dihtype(dih_i)

In [227]:
for dk,d in oplsaa.dihtypes.iteritems():
    print dk,d

0  dihedral  HC - CT - CT - HC type opls 
  k1 = 0.000000 k2 = 0.000000 k3 = 0.300000 k4 = 0.000000 lammps index 0  gromcas index 0 
1  dihedral  X - CT - CT - X type opls 
  k1 = 1.740000 k2 = -0.157000 k3 = 0.000000 k4 = 0.000000 lammps index 0  gromcas index 0 
2  dihedral  X - CA - CA - X type opls 
  k1 = 0.000000 k2 = 1.330000 k3 = 0.000000 k4 = 0.000000 lammps index 0  gromcas index 0 
3  dihedral  S - CA - CA - HA type opls 
  k1 = 0.000000 k2 = 10.750000 k3 = 0.000000 k4 = 0.000000 lammps index 0  gromcas index 0 


In [ ]:
lmp_i.set_ffparam()

Okay we have the monomer, so let's make a pentamer 

In [93]:
penta_ht = copy.deepcopy(ht)

In [96]:
# penta_ht = ht.prepattach('termcap',0,dir=-1,yangle=90.0)

In [94]:
for n in range(4):
    penta_ht = bb.attach(penta_ht,ht,'termcap',1,'termcap',0,tag='penta_3-hexyl-thiophene')

Check the charges of the removed hydrogens got summed onto the functionalized carbons correctly

In [102]:
penta_ht.calc_charge()
penta_ht.charge

-2.2551405187698492e-17

In [95]:
penta_ht.write_xyz()

Well it's cis, but we can run some high temperature MD to randomize that 

In [192]:
# del oplsaa.dihtypes[3]

In [ ]:
oplsaa_unit_conf = copy.deepcopy(CT.unit_conf)

In [ ]:
oplsaa_unit_conf['energy'] = 'kCalmol'

In [ ]:
pprint(oplsaa_unit_conf)

Sweet as bro!